In [ ]:
# importar bibliotecas
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# definir parâmetros
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-4
VAL_SPLIT = 0.2

# definir diretórios
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_DIR = os.path.join(BASE_DIR, 'data')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')
VAL_DIR = os.path.join(DATA_DIR, 'val')
MODEL_DIR = os.path.join(BASE_DIR, 'model')

# definir classes
CLASSES = ['with_mask', 'without_mask']

# definir geradores de imagens
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# definir geradores
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    seed=42
)

val_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    seed=42
)


In [ ]:
# definir modelo
base_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3))
)

head_model = base_model.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(128, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(1, activation='sigmoid')(head_model)

model = Model(inputs=base_model.input, outputs=head_model)

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=LR, decay=LR/EPOCHS),
    metrics=['accuracy']
)

model.summary()

In [ ]:
# definir callbacks
checkpoint = ModelCheckpoint(
    os.path.join(MODEL_DIR, 'mask_detector.model'),
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

callbacks = [checkpoint, earlystop]


In [ ]:
# treinar modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=callbacks
)

# avaliar modelo
predIdxs = model.predict(test_generator, batch_size=BATCH_SIZE)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(test_generator.classes, predIdxs, target_names=CLASSES))

cm = confusion_matrix(test_generator.classes, predIdxs)
total = sum(sum(cm))
accuracy = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

print(cm)
print('accuracy : ', accuracy)
print('sensitivity : ', sensitivity)
print('specificity : ', specificity)


In [ ]:
# plotar gráficos
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, EPOCHS), history.history['loss'], label='train_loss')
plt.plot(np.arange(0, EPOCHS), history.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, EPOCHS), history.history['accuracy'], label='train_acc')
plt.plot(np.arange(0, EPOCHS), history.history['val_accuracy'], label='val_acc')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='lower left')
plt.savefig(os.path.join(MODEL_DIR, 'plot.png'))


In [ ]:
# salvar modelo
model.save(os.path.join(MODEL_DIR, 'mask_detector.model'), save_format='h5')

# salvar classes
with open(os.path.join(MODEL_DIR, 'classes.txt'), 'w') as f:
    f.write('\n'.join(CLASSES))

# salvar parâmetros
with open(os.path.join(MODEL_DIR, 'params.txt'), 'w') as f:
    f.write('IMG_SIZE = ' + str(IMG_SIZE) + '

')
    f.write('BATCH_SIZE = ' + str(BATCH_SIZE) + '

')
    f.write('EPOCHS = ' + str(EPOCHS) + '

')
    f.write('LR = ' + str(LR) + '

')

# salvar histórico
with open(os.path.join(MODEL_DIR, 'history.txt'), 'w') as f:
    f.write(str(history.history))

# salvar métricas
with open(os.path.join(MODEL_DIR, 'metrics.txt'), 'w') as f:
    f.write('accuracy : ' + str(accuracy) + '

')
    f.write('sensitivity : ' + str(sensitivity) + '

')
    f.write('specificity : ' + str(specificity) + '

')

# salvar matriz de confusão
with open(os.path.join(MODEL_DIR, 'confusion_matrix.txt'), 'w') as f:
    f.write(str(cm))

# salvar relatório de classificação
with open(os.path.join(MODEL_DIR, 'classification_report.txt'), 'w') as f:
    f.write(str(classification_report(test_generator.classes, predIdxs, target_names=CLASSES)))

# salvar modelo em formato JSON
model_json = model.to_json()
with open(os.path.join(MODEL_DIR, 'model.json'), 'w') as json_file:
    json_file.write(model_json)

# salvar modelo em formato YAML
model_yaml = model.to_yaml()
with open(os.path.join(MODEL_DIR, 'model.yaml'), 'w') as yaml_file:
    yaml_file.write(model_yaml)

# salvar modelo em formato HDF5
model.save(os.path.join(MODEL_DIR, 'model.h5'))

# salvar modelo em formato SavedModel
tf.saved_model.save(model, os.path.join(MODEL_DIR, 'saved_model'))

# salvar modelo em formato TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(os.path.join(MODEL_DIR, 'model.tflite'), 'wb').write(tflite_model)

# salvar modelo em formato TensorFlow.js
tfjs.converters.save_keras_model(model, os.path.join(MODEL_DIR, 'tfjs_model'))

# salvar modelo em formato CoreML
coreml_model = coremltools.converters.keras.convert(model)
coreml_model.save(os.path.join(MODEL_DIR, 'model.mlmodel'))
